In [113]:
#torchvision.models 模型库操作
import torch
import torchvision.models as models

vit =  models.vision_transformer.vit_l_16(pretrained=True)

#打印整个模型和名字
# for (name, module) in vit.named_modules():
#     print(name)

#打印模型参数
# print(list(vit.encoder_layer_0.ln_1.named_parameters())[0])
# print(vit.state_dict()['encoder.layers.encoder_layer_0.ln_1.weight'])

# 截取模型中部分
# net2 = torch.nn.Sequential(*list(vit.modules())[:2])
# net2 = vit.encoder

print(vit)
# x = torch.rand(1,3,224,224)
# print(vit(x).shape)


VisionTransformer(
  (conv_proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (linear_1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU()
          (dropout_1): Dropout(p=0.0, inplace=False)
          (linear_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout_2): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine

In [ ]:
import torch
import torchvision.models as models

trans_block =  torch.nn.Sequential(*list(models.vision_transformer.vit_l_16(pretrained=True).encoder.layers.children())[:10])
# print(trans_block)
# print(trans_block)
# for (name, module) in trans_block.named_modules():
#     print(name)

print(list(trans_block[0].ln_1.named_parameters())[0])
# print(vit.state_dict()['trans_block.0.ln_1.weight'])
# print(trans_block.state_dict())

# print(list(trans_block.children())[:10])

# x = torch.rand(1,3,512,512)
# print(vit(x).shape)


In [3]:
#swin transformer
from typing import OrderedDict
import torch
from swin_transformer_pytorch import SwinTransformer

net = SwinTransformer(
    hidden_dim=256,
    layers=(2, 2, 6, 2),
    heads=(3, 6, 12, 24),
    channels=3,
    num_classes=3,
    head_dim=32,
    window_size=7,
    downscaling_factors=(4, 2, 2, 2),
    relative_pos_embedding=True
)
x = torch.randn(1, 3, 224, 224)
y = net(x)  # (1,3)
# print(net)
print(y.shape)

torch.Size([1, 3])


In [ ]:
#损失函数
import torch
import torch.nn as nn
 
cel = nn.CrossEntropyLoss()
nll = nn.NLLLoss()
logsoftmax = nn.LogSoftmax(dim=1)
 
x = torch.randn(3,3)
y_target = torch.tensor([1,2,0])
 
logsoftmax_value = logsoftmax(x)
nll_value = nll(logsoftmax_value,y_target)
cel_value = cel(x,y_target)
 
print('x',x)
print('y_target',y_target)
print('logsoftmax_value',logsoftmax_value)
print('nll_value',nll_value)
print('cel_value',cel_value)

In [ ]:
#获取模型中间层输出一（hook）
import torch,torchvision
from model.model_arc.backbone.VIT import *
import torch.nn as nn

net = ViT(
    image_size = 512,
    patch_size = 32,
    dim = 1024,
    depth= 1,
    heads=16,
    hid_dim=4096,
    dropout = 0.1,
    emb_dropout = 0.1)
C = torch.randn(1, 3, 512, 512)

# for k in net.state_dict():
#     print(k)
# for (name, module) in net.named_modules():
#     print(name)
# print(net.transformer.layers[1])

features_in_hook= []
features_out_hook = []
def forward_hook(module,data_input,data_output):
    features_in_hook.append(data_input)
    features_out_hook.append(data_output)

# net.transformer.register_forward_hook(hook=forward_hook)
(net.transformer.layers[0][0]).register_forward_hook(hook=forward_hook)
# (net.transformer.layers[1][0]).register_forward_hook(hook=forward_hook)
# (net.transformer.layers[0][1]).register_forward_hook(hook=forward_hook)
# (net.transformer.layers[0][0]).register_forward_hook(hook=forward_hook)
(net.transformer).register_forward_hook(hook=forward_hook)
(net.dropout).register_forward_hook(hook=forward_hook)


out = net(C)
a = features_out_hook
a

In [ ]:
#获取模型中间层输出一（IntermediateLayerGetter），只能获取一级模块
import torchvision
m = torchvision.models.resnet18(pretrained=True)
 # extract layer1 and layer3, giving as names `feat1` and feat2`
new_m = torchvision.models._utils.IntermediateLayerGetter(m,{'layer1': 'feat1', 'layer2': 'feat2', 'layer3': 'feat3', 'layer4': 'feat4'})
out = new_m(torch.rand(1, 3, 224, 224))
# print(m)
print([(k, v.shape) for k, v in out.items()])

In [23]:
from doctest import OutputChecker
import torch
import torch.nn as nn
#卷积操作
#kenel_size = 3 padding= 1 改变通道数操作
conv = nn.Sequential(nn.Conv2d(512, 256, 3, padding=1))
#输入(batch，channel，H_input，W_input)  输出(batch，channel，H_output，W_output)

#线性操作
liner = nn.Linear(512,256)
#输入(batch，channel，input)  输出(batch，channel，output)

#unfold操作
unfold = nn.Unfold(kernel_size=4, dilation=1, padding=0, stride=5)

#归一化操作
norm = nn.LayerNorm(1024)

#初始化方法
for m in conv.modules():
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
print(conv.state_dict())


# x = torch.rand(1,3,224,224)
# out = unfold(x)
# print(out.shape)

ValueError: Fan in and fan out can not be computed for tensor with fewer than 2 dimensions